In [10]:
# Importer packages
from bs4 import BeautifulSoup as bs
from requests import get
import re
import nltk

In [11]:
# Téléchargement des modules depuis nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\morcodou.seck\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\morcodou.seck\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Nettoyage du text

In [12]:
def cleaner(text_to_clean):
    # Supprimer [\w]*
    text_to_clean = re.sub(r'[[\w]*]', ' ', text_to_clean)

    # Supprimer les chaines de \xa0, \u200c
    text_to_clean = re.sub(r'\xa0|\u200c', ' ', text_to_clean)

    # Remplacer les espaces multiples par l'espace simple
    text_to_clean = re.sub(r'/s+', ' ', text_to_clean)

    # Remplacer l'espace en debut et fin de corpus
    text_to_clean = re.sub(r'^\s|\s$', '', text_to_clean)
    
    return text_to_clean

- Summarization

In [13]:
def calculate_word_weight(text, language = 'english'):
    # Stopwords
    stopwords = nltk.corpus.stopwords.words(language)
    
    # Dictionnaire de fréquences des mots
    word_frequencies = {}
    for word in nltk.word_tokenize(text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
                
    
    maximum_frequency = max(word_frequencies.values()) # Fréquence maximale
    # Calculer la fréquence pondérée
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / maximum_frequency

    return word_frequencies

In [14]:
def get_sentences_score(text, language = 'english'):
    sentence_list = nltk.sent_tokenize(text)
    word_frequencies = calculate_word_weight(text, language = language)
    sentence_scores = {} # Liste des scores de chaque phrase
    
    # Calculer le score de chaque phrase
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
                        
    return sentence_scores

In [15]:
def resume_text(text, resume_size, language = 'english'):
    sentence_scores = get_sentences_score(text, language = language)
    
    # Ordonner les phrases par pondération et recupérer les "resume_size" premières phrases
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=False)[:resume_size]
    
    # regrouper ensemble les phrases qui ont les poids les plus élévés
    summary = ' '.join(summary_sentences)

    # Afficher le résumé
    return summary

- Text summarization avec sumy

In [20]:
# Importer les packages
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [21]:
def with_textRankSummarizer(text, resume_size, language = 'english'):
    # Importer le TextRankSummarizer
    from sumy.summarizers.text_rank import TextRankSummarizer

    # Initialiser le modèle
    summarizer_textrank = TextRankSummarizer()
    
    # Créer un text parser utilisant de tokenisation
    parser = PlaintextParser.from_string(text, Tokenizer(language))

    # Summariser en 5 phrases
    summary = summarizer_textrank(parser.document, resume_size)

    # Regrouper les phrases
    text_summary = ""
    for sentence in summary:
        text_summary += str(sentence)

    # Retourner le summary
    return text_summary

In [22]:
def with_lexRankSummarizer(text, resume_size, language = 'english'):
    # Importer LexRankSummarizer
    from sumy.summarizers.lex_rank import LexRankSummarizer
    
    # Initialiser le modèle
    summarizer_lexrank = LexRankSummarizer()
    
    # Créer un text parser utilisant de tokenisation
    parser = PlaintextParser.from_string(text, Tokenizer(language))

    # Summariser en 5 phrases
    summary = summarizer_lexrank(parser.document, resume_size)

    # Regrouper les phrases
    text_summary = ""
    for sentence in summary:
        text_summary += str(sentence)
        
    # Afficher le summary
    return text_summary

In [26]:
def with_lsaSummarizer(text, resume_size, language = 'english'):
    # Importer LsaSummarizer
    from sumy.summarizers.lsa import LsaSummarizer
    
    # Initialiser le modèle
    summarizer_lsa = LsaSummarizer()
    
    # Créer un text parser utilisant de tokenisation
    parser = PlaintextParser.from_string(text, Tokenizer(language))

    # Summariser en 5 phrases
    summary = summarizer_lsa(parser.document, resume_size)

    # Regrouper les phrases
    text_summary = ""
    for sentence in summary:
        text_summary += str(sentence)

    # Afficher le summary
    return text_summary

In [52]:
# Testons tout ça 
text = """Mark Elliot Zuckerberg naît le 14 mai 1984 à White Plains dans l’État de New York au sein d’une famille juive américaine. Il est le fils de Karen Kempner (née en 1958), psychiatre, et d'Edward Zuckerberg (né en 1954), dentiste.

Du côté maternel, Karen Kempner est la fille de Sidney Kempner (1921-2012), fils de Harry Kempner (1892-1936) et Mary Steinberg (vers 1898-1965), et de Gertrude Silver (née en 1924).

Du côté paternel, Edward Zuckerberg est le fils de Jacob Zuckerberg (1919-2004), fils de Max Zuckerberg (1899-1948) et de Minnie Wiesenthal ( 1899-1986), et de Miriam Hollander (1920-2004), fille de Max Hollander (1873-1965) et de Rose Schoenfeld (1880-1957), originaires d’Autriche.

Avec ses trois sœurs Randi, Donna, et Arielle, Mark Zuckerberg est élevé à Dobbs Ferry, New York.

Son père, dentiste, est l'un des premiers à utiliser la radiographie numérique et possède plusieurs ordinateurs. Il lui offre un ordinateur Atari permettant de programmer en Basic. Entre 10 et 12 ans, il crée sa première messagerie, qu'il nomme ZuckNet, et permet à son père de communiquer instantanément avec ses collègues dentistes ou assistants, ainsi qu'avec sa famille. ZuckNet ressemble à une version primitive d'AOL Instant Messenger, sorti un an plus tard, en 1997. Entré au lycée, Mark Zuckerberg programme le logiciel Synapse, qui permet de déterminer les goûts musicaux des utilisateurs et de créer des listes d'écoute. Synapse est cité par des blogs technologiques. AOL et Microsoft veulent l'acquérir et recruter Mark Zuckerberg. Mais il décline leurs offres2,3,4,5. Bill Gates lui aurait offert un million de dollars pour rejoindre Microsoft6. Lorsqu'il entre à l'université Harvard en 2002, il a une réputation de prodige de la programmation. Au début de sa deuxième année, il crée le logiciel CourseMatch, qui permet aux étudiants de déterminer quels cours suivre. Peu après, il crée Facemash, qui permet de noter les étudiants les plus sexys à partir de leurs photos, une application rapidement fermée par l'université2.

Au collège, il s'est passionné pour le latin7, et a étudié l'antiquité grecque et romaine8. Il maîtrise l'hébreu, le français, le latin et le grec ancien, et s'est initié au mandarin4,9,5."""
resume_size = 2
language = "french"

In [53]:
custom_summury = resume_text(text = text, resume_size = resume_size, language = language)
textRank_summury = with_textRankSummarizer(text = text, resume_size = resume_size, language = language)
lexRank_summury = with_lexRankSummarizer(text = text, resume_size = resume_size, language = language)
lsa_summury = with_lsaSummarizer(text = text, resume_size = resume_size, language = language)

In [57]:
# custom_summury
# textRank_summury
# lexRank_summury
lsa_summury

'Mark Elliot Zuckerberg naît le 14 mai 1984 à White Plains dans l’État de New York au sein d’une famille juive américaine.Avec ses trois sœurs Randi, Donna, et Arielle, Mark Zuckerberg est élevé à Dobbs Ferry, New York.'